# PLZ to Index
*Thomas Bissinger, Hack and Harvest KN, 29.06.24*

In a first step, this script takes postal codes and turns them into latitude and longitude data. The resulting latitude and longitude is then converted into indices in the matrix representation in the COPERNICUS heat wave dataset. Uses the opencage package to extract latitudes and longitudes, an **api key** is required to run this script. There is a free subscription available. The code is partially in German.

**Attention**: The indexing convention in this script is NOT used in the .nc file

The data set can be found under *https://cds.climate.copernicus.eu/cdsapp#!/dataset/sis-heat-and-cold-spells?tab=overview*

In [1]:
%pip install -q opencage
%pip install -q requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Simple Example
To get started, let us perform a simple latitude/longitude extraction.

In [2]:
# A simple implementation to extract geocode
from opencage.geocoder import OpenCageGeocode
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
# OpenCage API key
api_key = 'API KEY REQUIRED'
geocoder = OpenCageGeocode(api_key)

# postcode / PLZ
postcode = '78462'  # 78462: Konstanz

# request to geocoding API
query = f'{postcode}, Germany'
results = geocoder.geocode(query)

# Check results
if results:
    latitude = results[0]['geometry']['lat']
    longitude = results[0]['geometry']['lng']
    print(f'Längen- und Breitengrad für {postcode}:')
    print(f'Breitengrad: {latitude}')
    print(f'Längengrad: {longitude}')
else:
    print('Keine Ergebnisse gefunden')


Längen- und Breitengrad für 78462:
Breitengrad: 47.6619501
Längengrad: 9.1706791


## Latitude and Longitude Extraction Script
Now, we do what we just did within a script.

In [3]:
# Function that returns longitude and latitude for a given post code
def get_geocode(postcode, country_code):
    # Importing requests module within the function
    import requests
    
    # Importing json module within the function
    from opencage.geocoder import OpenCageGeocode as ocg
    
    # OpenCage API key
    api_key = 'API KEY REQUIRED'
    
    # Base URL for the OpenCage Geocoding API
    base_url = 'https://api.opencagedata.com/geocode/v1/json'
    
    # Parameters to be sent to the API
    params = {
        'q': postcode,
        'key': api_key,
        'countrycode': country_code
    }
    
    # Make a request to the API
    response = requests.get(base_url, params=params)
    
    # Check the status code of the response
    if response.status_code == 200:
        # Parse the JSON data from the response
        data = response.json()
        if data['results']:
            # Extract latitude and longitude from the first result
            latitude = data['results'][0]['geometry']['lat']
            longitude = data['results'][0]['geometry']['lng']
            return (latitude, longitude)
        else:
            print('No results found')
            return None
    else:
        print('Failed to retrieve data:', response.status_code)
        return None

## Matrix Index Extraction Script
We then need a script that extracts the two matrix indices associated with a latitude and longitude

In [4]:
# For a given longitude and latitude, returns the matrix indices of the grid point in the bounding box (heatwave data set)
def find_matrixindex(latitude, longitude):
    import math

    if latitude == None or longitude == None:
        print('Invalid Input')
        return None
    elif latitude > 72.6 or latitude < 25:
        print('Latitude out of bounds. Bounds are (25,72.6), latitude is ', latitude)
        return None
    elif longitude > 35 or longitude < -25:
        print('Latitude out of bounds. Bounds are (25,75.6), latitude is ', latitude)
        return None
    else:
        ref_latitude = 72.60
        ref_longitude = -25.00
        step_latitude = -0.10
        step_longitude = 0.10
        index_x = math.floor((longitude - ref_longitude) / step_longitude)
        index_y = math.floor((latitude - ref_latitude) / step_latitude)
        return (index_x, index_y)

## Final Example
Simple example code for using the two scripts just defined to convert a postcode into two matrix indices

In [5]:
# Example usage of the function
postcode = '78462' # '80336' for Munich, '78462' for Konstanz
country_code = 'de'
coordinates = get_geocode(postcode, country_code)
if coordinates:
    indices = find_matrixindex(coordinates[0],coordinates[1])
    print(f'Postcode {postcode}:')
    print(f'Latitude: {coordinates[0]}')
    print(f'Longitude: {coordinates[1]}')
    print(f'x index: {indices[0]}')
    print(f'y index: {indices[1]}')

Postcode 78462:
Latitude: 47.6619501
Longitude: 9.1706791
x index: 341
y index: 249
